In [3]:
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from enum import Enum, auto
from random import uniform, random
from math import cos, sin, pi, inf, sqrt
import os

this_dir = "C:\\Users\\na366\\Desktop\\Python\\SIR"
radius = 0.075
dt = 0.01        # time_to_recover = 0.1/0.01 = 10 時間indexで,I ⇒ R
half = 0.2                # half =5
speed = 2                # speed = 1
time_to_recover = 0.1    # time_to_recover =2
buff = 1                # buff = 300

#                                            図.
fig = plt.figure(figsize=plt.figaspect(1/1))
#                                            図の背景色は黒.
fig.set_facecolor('black')


class State(Enum):
    Susceptible = auto()
    Infected = auto()
    Recovered = auto()

class Person:
    
    def __init__(self,co,state,standstill):
        
        # 93_self.persons.append(Person(co,state,standstill))
        self.co = co
        self.coords = np.array([self.co])    # 座標列[co_0,co_1,co_2,,]
        self.arg = uniform(-pi, pi)
        self.standstill = standstill    # 停止　True-or-Furse
        if self.standstill:
            vel = np.array([0.0, 0.0])    # 停止　True=vel[x_0,y_0]
        else:
            vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])
        self.vel = vel        # [0,0]_or_[x_arg,y_arg]
        self.state = state    # S,I,R
        
        # overflow,（1 / 0）、log(0) Infinity
        self.infection_start = inf    # 感染源 I_start_i(時間index)
        
    def move(self):
        if self.standstill:
            self.vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])
            
class Population:
    
    def __init__(self,number,comeback,p=None):
        
        # 230_Population(numbe =10   ,comeback =True,  p =0.5)
        self.number = number
        self.R0 = (radius * (sqrt(2) * speed) * time_to_recover * self.number) / (half ** 2)
        print("R0: %f" % self.R0)
        self.persons = []
        self.p = p
        init_coords = []
        if self.p is not None:
            self.Re = (1.0 - self.p) * (1.0 + (sqrt(2) - 1) * self.p) * self.R0
            print("Re: %f" % self.Re)
        for i in range(self.number):    # [0,1,2,,]順に代入
            if i == 0:
                init_co = np.array([0.0, 0.0])
                
                infected = Person(co=init_co,state=State.Infected,standstill=False)
                # P1 ← 代入object_P1(variable_7)_class_return ← class引数(座標,感染 I ,停止ではない)
                
                # P1(variable_infection_start)←0_代入
                infected.infection_start = 0    # 感染源 I_start_i=0(時間index)計数始
                
                # object列[0,1,,] ← object_P1 追記
                self.persons.append(infected)
                
            else:
                whi=0
                while True:
                    init_co = np.array([uniform(-half, half), uniform(-half, half)])
                    appropriate = True
                    foi=0
                    for pre_init_co in init_coords:
                        dist = np.linalg.norm(init_co - pre_init_co, ord=2)
                        if dist <= 3 * radius:
                            # print("72___dist-3*radius=",dist-3*radius)
                            appropriate = False
                            
                        if appropriate == False:
                            # print("76_Population_i-whi-for_break=",i,"-",whi,"-",foi,"\n")
                            break
                            
                        foi += 1
                        
                    if appropriate:    # False,は,while(座標作,重り)繰返し
                        break
                        
                    whi += 1
                    
                if self.p is None:
                    standstill = False
                elif random() < self.p:
                    standstill = True
                else:
                    standstill = False
                self.persons.append(Person(co=init_co,
                                           state=State.Susceptible,
                                           standstill=standstill))
                # print("102_i ,whi, persons)= ",i,",",whi,",",self.persons,"\n")
                
            init_coords.append(init_co)
            # print("105_i,init_coords= ",i,",",init_coords,"\n")

        self.times = np.array([0])
        self.zeros = np.array([0])
        self.I = np.array([1])
        self.S = np.array([self.number - 1])
        self.R = np.array([0])
        self.comeback = comeback
        
        # print("114_Population_i,len(persons)",i,",",len(self.persons),"\n")
        # print("115_Population_vars(self.persons[3])=",vars(self.persons[3]),"\n")


    def simulate_sir(self,threshold=2):    # 閾値=2
        
        i = 0    # i=時間index                   超えた(exceeded)
        exceeded_threshold = False    # Iが閾値（threshold）を超えたか
        comebacked = False    # 行動制限を解除したか否か.
        subsided = None    # 感染が収束した時の時間のインデックス.
        
        # 感染収束 subsided + バッファ =時間インデックス ⇒ def_simulate_sir
        
        while True:
            # 感染始 I⇒回復時間 time_to_recover⇒免疫獲得 R 動再開
            # 正方形 Box 壁に当り 反射
            
            for person in self.persons:    # [列,object[0],object[1],[2]代入
                # (72)_i=0_infection_start=0,他=inf
                # print("時間index_i=",i,", for_i=",person,"
                # , 142_person.infection_start=",person.infection_start)
                # print("時間index_i=",i,", for_i=",person,"
                # , 143_i-person.infection_start=",i - person.infection_start)
                
                if (i - person.infection_start) * dt > time_to_recover:
                    # time_index - object_要素(感染始time_index)*0.01> 0.05
                    person.state = State.Recovered    # 要素state_I⇒R
                    
                # vel[x,y]×dt=[xdt,ydt]=Δvel ,座標値 co+Δvel
                person.co += dt * person.vel
                # ｘ+Δxで、壁反射.
                if abs(person.co[0]) > half:
                    person.co[0] = np.sign(person.co[0]) * half
                    person.vel[0] *= -1
                # ｙ+Δｙで、壁反射.
                if abs(person.co[1]) > half:
                    person.co[1] = np.sign(person.co[1]) * half
                    person.vel[1] *= -1
                    
            indices = list(range(self.number))        # indices_索引
            print("163_indices = list(range(self.number))",indices)    # [0, 1, 2]
            
            whi=0
            comebacke_i_point=0
            
            while len(indices) > 0:    # []でない ⇒ []で,while_break
                
                # pop()の引数を空白にすると末尾の要素が削除
                # 戻値は削除した要素
                # 指定インデックス番号の要素が削除
                k = indices.pop(0)
                
                # 元のリストや辞書の,要素が削除,indices= [1, 2,]
                print("176_t_i=",i,", whi=",whi
                      ," indices.pop(0)削除値が戻値=",k,", ",indices)
                # 削除した_k_の_object[k]は、ヴぁｒｓ
                person = self.persons[k]
                # 先頭_1番_object_persons[0](vars(_7要素)
                # print("181_i=",i,", whi=",whi,", numberl=",l,", persons[",k,"]= ",person)
                # print("182_i=",i,", whi=",whi,", numberl=",l,", len(self.persons)= ",len(self.persons))
                
                for l in indices:            # [1, 2]
                    print("185_t_i=",i,", whi=",whi,", n_l=",l
                          ,", indices=",indices) 
                    
                    other = self.persons[l]    # 2番_object
                    print("189_i=",i,", whi=",whi,", for-in_persons[",l,"]= ",other)
                    
                    vec = person.co - other.co
                    dist = np.linalg.norm(vec, ord=2)
                    print("193_(先頭-次)=L2を,順に,for_l=",l
                          ,", person=",person,", other=",other,", vec=",vec,", dust=",dist)
                    
                    # person と other の距離が 2 *（半径）より小さいとき
                    # 接触したと判定.3 人以上の同時接触は考えない.
                    
                    if 0 < dist < 2 * radius:
                        print("\n199_ for_0 < dist < 2 * radius:")
                        
                        normal = vec / dist
                        # print("\n","i=",i,", whi=",whi,", numberl=",l,"
                        # , 190_normal = vec/dist ",normal,"=",vec,"/",dist)
                        
                        # person と other が重ならないようにするための補正
                        # （correction）.
                        correction = (radius - (dist / 2)) * normal
                        # print("i=",i,", whi=",whi,", numberl=",l,", 195_補正=",correction)
                        pc1=person.co
                        person.co += correction
                        # print("i=",i,", whi=",whi,", numberl=",l,", 198_person.co=",pc1,"
                        # , person.co + 補正=",person.co)
                        
                        other.co -= correction
                        # print("i=",i,", whi=",whi,", numberl=",l,", 201_other.co-補正=",other.co)
                        
                        # print("i=",i,", whi=",whi,", numberl=",l,", 203_person.vel=",person.vel,"
                        # , normal=",normal)
                        # print("i=",i,", whi=",whi,", Inumberl=",l,", 204_dot(person.vel,normal)_内積=
                        #                                             ",np.dot(person.vel,normal))
                        
                        # 物理的な完全弾性衝突の場合は -= np.dot(person.vel - other.vel, normal) * normal.
                        person.vel -= (2 * np.dot(person.vel, normal)) * normal
                        # print("i=",i,", whi=",whi,", numberl=",l,", 208_person.vel=",person.vel)
                        
                        # 物理的な完全弾性衝突の場合は -= np.dot(other.vel - person.vel, normal) * normal.
                        other.vel -= (2 * np.dot(other.vel, normal)) * normal
                        # print("i=",i,", whi=",whi,", numberl=",l,", 212_other.vel=",other.vel)
                        
                        #  160_ indies=[0, 1, 2]
                        # リストのメソッドremove()で、指定した値と同じ要素を検索し、最初の要素を削除
                        print("224_indices=",indices)
                        indices.remove(l)
                        print("221_indices.remove(",l,"検索削除),  t_i=",i
                              ,", whi=",whi,", n_l=",l,", indices=",indices)
                        
                        # 感染状態を必要であれば変化させる.
                        if person.state == State.Infected and other.state == State.Susceptible:
                            other.state = State.Infected
                            other.infection_start = i
                            print("241_t_i=",i,", whi=",whi,", n_l=",l
                                  ,", other.state=",other.state.value
                                  ,"other.infection_start=",other.infection_start)
                        
                        elif person.state == State.Susceptible and other.state == State.Infected:
                            person.state = State.Infected
                            person.infection_start = i
                            print("248_t_i=",i,", whi=",whi
                                  ,"n_l=",l,", person.state=",person.state
                                  ,"person.infection_start=",person.infection_start)
                            
                        print("251_if_接触_break, t_i=",i,", whi=",whi,", n_l=",l,"\n")
                        break
                        
                        
                    # if_
                    print("256_t_i=",i,", whi=",whi,", n_l=",l,"for_in_indices=",indices) 
                # for_in
                # 173 _indices.pop(0)_で_while_条件_len(indices)>0_Fulseになる
                print("259_t_i=",i,", whi=",whi," indices=",indices," ← while len(indices) > 0:\n")    
                if indices == []:
                    print("----")
                whi+=1
                
            # print("i=",i,"__218_時間index")
            # while_時間の index を一つ進める
            i += 1
            
            I = 0    # 総感染者数
            S = 0    # 総未感染者数
            R = 0    # 総免疫獲得者数
            foi=0
            for person in self.persons:
                
                # 各人間に対し, 現在の座標を追加記録させる.
                # 2次元縦方向(vertical),axis=0,行方向連結、上0、下1、2、最下位に追加
                # print("i=",i,", foi=",foi," , 229_現在の座標_person.co= ",person.co)
                
                person.coords = np.vstack([person.coords, person.co])
                # print("i=",i,", foi=",foi," , 232_行方向連結で追加記録座標
                # person.coords= \n",person.coords)
                # print("i=",i,", foi=",foi," , 233_len(person.coords)= ",len(person.coords))
                # print("i=",i,", foi=",foi," , 234_len(vars(person))= ",len(vars(person)))
                # print("i=",i,", foi=",foi," , 272_person.state(S/1,I/2,R/3)= ",person.state.value)
                if person.state == State.Infected:
                    I += 1
                elif person.state == State.Susceptible:
                    S += 1
                elif person.state == State.Recovered:
                    R += 1
                # print("i=",i,", foi=",foi,", 279_S,I,R=",S,",",I,",",R,)
                foi+=1
                    
            # 3,総感染者数 I > 感染者数の閾値（threshold =2）
            #       = exceeded_threshold =True.
            
            if threshold < I and not exceeded_threshold:
                exceeded_threshold = True
                
            # 行動制限を解除する場合
            # 1,self.comeback =True
            # 2,行動制限中 comebacked =Furse
            # 3,exceeded_threshold =True
            # その後
            # 1,総感染者数 I <= 感染者数の閾値 threshold = 2
            # 2,停止させている人間は全て動かす ← 行動制限を解除
            
            # print("232_",I <= threshold,self.comeback,not comebacked,exceeded_threshold)
            comebacked_1=I <= threshold and self.comeback and not comebacked and exceeded_threshold
            
            if comebacked_1:
                if comebacke_i_point > i:
                    break
                for person in self.persons:
                    person.move()
                comebacked = True
                # print("257_comebacke=True_i=",i,"\n")
                comebacke_i_point=i
                
                
            # I=1 開始 ⇒ I=0 感染収束,時間 index_i = subsided_沈静化
            
            if I == 0 and subsided is None:
                subsided = i
            # if comebacked and not I==0:
                # print("265_Simulating SIR %d (S: %d, I: %d, R: %d) (Comebacked)" % (i, S, I, R))
            if I==0:
                print("i=",i,"312_Simulating SIR %d (S: %d, I: %d, R: %d)" % (i, S, I, R),"\n")
                
            # それぞれの数値計算結果を記憶.
            self.times = np.append(self.times, i * dt)
            self.zeros = np.append(self.zeros, 0)
            self.I = np.append(self.I, I)
            self.S = np.append(self.S, S)
            self.R = np.append(self.R, R)
            
            # 感染収束時の時間の index が subsided に記憶されており,
            # 時間の index（= i）= subsided +バッファ
            #  ⇒ シミュレーションを終了 break
            
            if subsided is not None and i == subsided + buff:
                self.end = i
                break
                
if __name__ == "__main__":
    population = Population(number=3 ,comeback=True,p=0.5)
    population.simulate_sir()
    # population.animate()
    


R0: 1.590990
Re: 0.960248
163_indices = list(range(self.number)) [0, 1, 2]
176_t_i= 0 , whi= 0  indices.pop(0)削除値が戻値= 0 ,  [1, 2]
185_t_i= 0 , whi= 0 , n_l= 1 , indices= [1, 2]
189_i= 0 , whi= 0 , for-in_persons[ 1 ]=  <__main__.Person object at 0x000002679C45C040>
193_(先頭-次)=L2を,順に,for_l= 1 , person= <__main__.Person object at 0x000002679E655520> , other= <__main__.Person object at 0x000002679C45C040> , vec= [ 0.18263535 -0.17546924] , dust= 0.25326887729309755
256_t_i= 0 , whi= 0 , n_l= 1 for_in_indices= [1, 2]
185_t_i= 0 , whi= 0 , n_l= 2 , indices= [1, 2]
189_i= 0 , whi= 0 , for-in_persons[ 2 ]=  <__main__.Person object at 0x000002679E655550>
193_(先頭-次)=L2を,順に,for_l= 2 , person= <__main__.Person object at 0x000002679E655520> , other= <__main__.Person object at 0x000002679E655550> , vec= [-0.1596045 -0.1364636] , dust= 0.20999026000183268
256_t_i= 0 , whi= 0 , n_l= 2 for_in_indices= [1, 2]
259_t_i= 0 , whi= 0  indices= [1, 2]  ← while len(indices) > 0:

176_t_i= 0 , whi= 1  indices.

193_(先頭-次)=L2を,順に,for_l= 2 , person= <__main__.Person object at 0x000002679E655520> , other= <__main__.Person object at 0x000002679E655550> , vec= [ 0.01497237 -0.16909129] , dust= 0.1697528647545413
256_t_i= 12 , whi= 0 , n_l= 2 for_in_indices= [1, 2]
259_t_i= 12 , whi= 0  indices= [1, 2]  ← while len(indices) > 0:

176_t_i= 12 , whi= 1  indices.pop(0)削除値が戻値= 1 ,  [2]
185_t_i= 12 , whi= 1 , n_l= 2 , indices= [2]
189_i= 12 , whi= 1 , for-in_persons[ 2 ]=  <__main__.Person object at 0x000002679E655550>
193_(先頭-次)=L2を,順に,for_l= 2 , person= <__main__.Person object at 0x000002679C45C040> , other= <__main__.Person object at 0x000002679E655550> , vec= [-0.24460556 -0.02875579] , dust= 0.2462900231489296
256_t_i= 12 , whi= 1 , n_l= 2 for_in_indices= [2]
259_t_i= 12 , whi= 1  indices= [2]  ← while len(indices) > 0:

176_t_i= 12 , whi= 2  indices.pop(0)削除値が戻値= 2 ,  []
259_t_i= 12 , whi= 2  indices= []  ← while len(indices) > 0:

----
163_indices = list(range(self.number)) [0, 1, 2]
176_t_i= 13 

<Figure size 288x288 with 0 Axes>

In [ ]:
"""
i= 10 , foi= 2 240_S,I,R= 2 , 1 , 0 
i= 11 , foi= 2 240_S,I,R= 1 , 2 , 0 
i= 22 , foi= 2 240_S,I,R= 1 , 1 , 1 
i= 32 , foi= 2 240_S,I,R= 1 , 0 , 2 
i= 33 277_Simulating SIR 33 (S: 1, I: 0, R: 2) 
"""

In [ ]:
"""
R0: 5.303301
Re: 3.200825
114_Population_i,len(persons) 49 , 50 

181_whi= 49
186_時間index, i= 0
218_ True True True False
Simulating SIR 1 (S: 49, I: 1, R: 0)
          　　　　　　　　 ↑
subsided= None buff= 1 

-------------

186_時間index, i= 176
218_ False True True True
Simulating SIR 177 (S: 18, I: 27, R: 5)
subsided= None buff= 1 
-------
186_時間index, i= 178
218_ False True True True
Simulating SIR 179 (S: 16, I: 29, R: 5)
          　　　　　　　　　　↑
subsided= None buff= 1 
------------
186_時間index, i= 180
218_ False True True True
Simulating SIR 181 (S: 16, I: 28, R: 6)
subsided= None buff= 1

-----------------------------------------

181_whi= 49
186_時間index, i= 326
218_ False True True True
Simulating SIR 327 (S: 7, I: 3, R: 40)
subsided= None buff= 1 

186_時間index, i= 327
218_ True True True True
Simulating SIR 328 (S: 7, I: 2, R: 41) (Comebacked) 
         　　　　　　　　　 ↑
--------------------------------

186_時間index, i= 558
218_ True True False True
Simulating SIR 559 (S: 3, I: 1, R: 46) (Comebacked) 

subsided= None buff= 1 
          ↑
186_時間index, i= 559
218_ True True False True
Simulating SIR 560 (S: 3, I: 0, R: 47) (Comebacked) 

subsided= 560 buff= 1 
          ↑
186_時間index, i= 560
218_ True True False True
Simulating SIR 561 (S: 3, I: 0, R: 47) (Comebacked) 

subsided= 560 buff= 1 


"""